In [5]:
import pandas as pd
import numpy as np
import re

In [3]:
path= './data/'
columnas = ['Score', 'Title', 'Review']
df=pd.read_csv(path+'AmazonReview.zip', compression='zip', header=None, names=columnas)
df

,Score,Title,Review
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...
...,...,...,...
399995,1,Unbelievable- In a Bad Way,We bought this Thomas for our son who is a hug...
399996,1,"Almost Great, Until it Broke...",My son recieved this as a birthday gift 2 mont...
399997,1,Disappointed !!!,"I bought this toy for my son who loves the ""Th..."
399998,2,Classic Jessica Mitford,This is a compilation of a wide range of Mitfo...


In [6]:
def clean_doc(doc):
    doc = re.sub(pattern=r'<.*?>', repl='', string=doc)
    doc = re.sub(pattern=r'[^A-Za-z0-9\-\']+', repl=' ', string=doc)
    doc = doc.lower()
    return re.sub(r'\s+', ' ', doc).strip()
def tokenizar(review):
    return review.split()

In [7]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def aplicar_stemming(tokens):
    return [stemmer.stem(token) for token in tokens]

def aplicar_lematizacion(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]


In [8]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\asus12\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\asus12\AppData\Roaming\nltk_data...


True

In [9]:
df_clean = df["Review"].apply(clean_doc)

In [14]:
df_10 = df_clean.head(10000)

In [15]:
df_stem = df_10.apply(lambda x: aplicar_stemming(tokenizar(x)))

In [16]:
df_stem

0       [my, love, pat, ha, one, of, the, great, voic,...
1       [despit, the, fact, that, i, have, onli, play,...
2       [i, bought, thi, charger, in, jul, 2003, and, ...
3       [check, out, maha, energy', websit, their, pow...
4       [review, quit, a, bit, of, the, combo, player,...
                              ...                        
9995    [thi, wa, a, wonder, read, on, poetri, and, i,...
9996    [the, devic, work, well, enough, but, i, am, d...
9997    [my, daughter, love, thi, she, is, six, month,...
9998    [i, purchas, thi, item, for, my, 6, month, old...
9999    [my, daughter, start, enjoy, thi, around, 4-1,...
Name: Review, Length: 10000, dtype: object

In [17]:
df_lemm = df_10.apply(lambda x: aplicar_lematizacion(tokenizar(x)))

In [18]:
df_lemm

0       [my, lovely, pat, ha, one, of, the, great, voi...
1       [despite, the, fact, that, i, have, only, play...
2       [i, bought, this, charger, in, jul, 2003, and,...
3       [check, out, maha, energy's, website, their, p...
4       [reviewed, quite, a, bit, of, the, combo, play...
                              ...                        
9995    [this, wa, a, wonderful, read, on, poetry, and...
9996    [the, device, work, well, enough, but, i, am, ...
9997    [my, daughter, love, this, she, is, six, month...
9998    [i, purchased, this, item, for, my, 6, month, ...
9999    [my, daughter, started, enjoying, this, around...
Name: Review, Length: 10000, dtype: object